### Setup

In [94]:
import pulp
import random, re
import numpy as np
import itertools
import functools
import collections
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill
import math
%matplotlib inline
plt.style.use('seaborn')
from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"
# %config InlineBackend.figure_format = 'retina'

pd.set_option('display.max_columns', 100)

### Parameters

In [95]:
# Number of employees to plan
n_employees = 13
# Number of weeks to plan
n_weeks = 4
n_days = 5 * n_weeks

# Maximum number of employees used as opt variables
max_empl_members = 4
# Maximum number of consecutive days used as opt variables
max_cons_days = 2
# Maximum and minimum number of people on the same day
max_people_per_day = 6
min_people_per_day = 0
# Define the target number of days one person should come in the office
target_presence = 2 * n_weeks

# Define the teams as tuples
teams = (
        (1, 2, 3),
        (4, 5, 6),
        (8, 9, 10),
        (11, 12)
)
assert(all(1 < len(x) <= max_empl_members for x in teams)), "Not supported length!"

# Start end cells describing availabilities
start_av, end_av = 'A2', 'AX14'

In [96]:
employees_ids = range(1, n_employees + 1)
days_ids = range(1, n_days + 1)
date_range = pd.date_range(start='10-18-2021', periods = n_days, freq='B')

#### Load data

In [97]:
empls = 'preferences3.xlsx' 
wb = load_workbook(empls, data_only = True)
sh = wb['Foglio1']

availabilities = {}

available, unavailable = 'FF92D050', 'FFFF0000'

employees_names = []

rows = sh[start_av: end_av]
for empl, row in enumerate(rows):
    availabilities[empl + 1] = []
    employees_names.append(row[0].value)
    for cell in row[1:]:
        color_in_hex = cell.fill.start_color.index
        if color_in_hex == available:
            availabilities[empl + 1].append(1)
        elif color_in_hex == unavailable:
            availabilities[empl + 1].append(0)
        else:
            availabilities[empl + 1].append(0)

# Delete or add extra elements
# if the excel file does not have or have too much columns
availabilities = {key: val[:n_days] + [0]*(n_days - len(val)) for key, val in availabilities.items()}
# availabilities

In [98]:
av = pd.DataFrame(availabilities).T
av.index = employees_names
av.columns = date_range.strftime('%b %d')
display(av.astype(int).style.background_gradient(cmap="Blues"))

,Oct 18,Oct 19,Oct 20,Oct 21,Oct 22,Oct 25,Oct 26,Oct 27,Oct 28,Oct 29,Nov 01,Nov 02,Nov 03,Nov 04,Nov 05,Nov 08,Nov 09,Nov 10,Nov 11,Nov 12
BISTECCA,1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,0
BURRO,1,1,1,1,1,1,1,0,1,1,0,0,1,1,0,0,1,1,0,1
FILETTO,1,1,0,1,1,1,1,0,0,1,1,1,0,1,0,0,0,1,0,0
FORMAGGIO,0,1,0,1,0,0,1,0,0,1,1,1,1,1,0,1,1,1,1,1
GELATO,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0
INSALATA,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0
LIMONE,1,1,1,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1
ORATA,0,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,1,1,1,0
SALMONE,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,0
STRUTTO,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1


### Optimization

#### Model variables

In [99]:
def window(seq, n = 2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(itertools.islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [100]:
x = {}
# Generate combinations of (consecutive) days variables
for empl in employees_ids:
    for day in range(1, max_cons_days + 1):
        cons = window(days_ids, day)
        for product_var in cons:
            name = [f'{empl}_{y}' for y in product_var]
            name_var = [f'x_{empl}_{y}' for y in product_var]
            x['__'.join(name)] = pulp.LpVariable('__'.join(name_var), cat = 'Binary')

# Generate combinations of employee variables
for day in days_ids:
    for empl in range(1, max_empl_members + 1):
        combs = list(itertools.combinations(employees_ids, empl))
        for product_var in combs:
            name = [f'{y}_{day}' for y in product_var]
            name_var = [f'x_{y}_{day}' for y in product_var]
            x['__'.join(name)] = pulp.LpVariable('__'.join(name_var), cat = 'Binary')
    
y = {}
for empl in employees_ids:
    y[str(empl)] = pulp.LpVariable(f'y_{empl}', cat = 'Integer')
    
    
print('x variables {}'.format(len(x)))
print('y variables {}'.format(len(y)))

x variables 22087
y variables 13


#### Objective

In [101]:
def get_employees_vars_all_inset(d, s):
    """
    Retrieve all the variables related
    to the employees from dictionary d.
    A variable is selected if all its terms 
    are in set
    """
    empls = {}
    for key, value in d.items():
        splitted = key.split('__')
        for el in splitted:
            if int(el.split('_')[0]) not in s:
                break
        else:
            empls[key] = value
    return empls


def get_employee_vars_all(d, employee):
    """
    Retrieve all the variables involving
    the employee from dictionary d.
    A variable is selected if all of its terms
    involve employee.
    """
    empl = {}
    for key, value in d.items():
        splitted = key.split('__')
        for el in splitted:
            if int(el.split('_')[0]) != employee:
                break
        else:
            empl[key] = value
    return empl


def get_day_vars_all(d, day):
    """
    Retrieve all the variables involving
    the day from dictionary d.
    A variable is selected if all of its terms
    involve day.
    """
    days = {}
    for key, value in d.items():
        splitted = key.split('__')
        for el in splitted:
            if int(el.split('_')[-1]) != day:
                break
        else:
            days[key] = value
    return days
          
    
def get_consecutive_days_vars(d):
    """
    Retrieve consecutive days variables from 
    dictionary d. A variable is selected if
    it involves consecutive days
    """
    cons = {}
    for key, value in d.items():
        splitted = key.split('__')
        if len(splitted) > 1:
            first = int(splitted[0].split('_')[-1])
            for el in splitted[1:]:
                if first + 1 != int(el.split('_')[-1]):
                    break
                first = int(el.split('_')[-1])
            else:
                cons[key] = value
    return cons 


def get_length_vars(d, length):
    """
    Return the variables having chosen
    lenght
    """
    lng = {}
    for key, value in d.items():
        splitted = key.split('__')
        if len(splitted) == length:
            lng[key] = value
    return lng


def get_not_crossweek_vars(d):
    """
    Return the variables which 
    do not cross the weekend
    """
    uncross = {}
    for key, value in d.items():
        splitted = key.split('__')
        if len(splitted) > 1:
            first = int(splitted[0].split('_')[-1])
            cross = False
            for el in splitted[1:]:
                # There is a 6 in here because variables are
                # indexed starting from 1
                if first // 6 != int(el.split('_')[-1]) // 6:
                    cross = True
                first = int(el.split('_')[-1])
            if not cross:
                uncross[key] = value
        else:
            uncross[key] = value
    return uncross 

In [102]:
objective = 0
cons_term = 0
single_returns_term = 0
target_presence_term = 0
team_term = 0

# Will be used for linearizing
linearize_vars = {}

for empl in employees_ids:
    
    # Add consecutive days bonus
    employee_vars = get_employee_vars_all(x, empl)
    consecutive_vars = get_consecutive_days_vars(employee_vars)
    two_length_vars = get_length_vars(consecutive_vars, 2)
    uncrossed_vars = get_not_crossweek_vars(two_length_vars)
    cons_term += pulp.lpSum(uncrossed_vars)
    linearize_vars.update(uncrossed_vars)
    
    # Add single returns bonus
    employee_vars = get_employee_vars_all(x, empl)
    single_vars = get_length_vars(employee_vars, 1)
    single_returns_term += pulp.lpSum(single_vars)

# Add teams bonus
for team in teams:
    for day in days_ids:
        day_vars = get_day_vars_all(x, day)
        team_vars = get_employees_vars_all_inset(day_vars, team)
        for i in range(len(team), 1, -1):
            i_length_vars = get_length_vars(team_vars, i)
            sign = -1 if i % 2 == 1 else 1
            team_term += + sign * pulp.lpSum(i_length_vars)
            linearize_vars.update(i_length_vars)
    

# Add target presence penalty
target_presence_term = pulp.lpSum(y)
    
# objective = single_returns_term
objective = 2*target_presence_term + cons_term  + team_term
# objective = team_term + target_presence_term

#### Constraints

In [103]:
constraints = []

# A maximum number of people each day
for day in days_ids:
    day_vars = get_day_vars_all(x, day)
    single_vars = get_length_vars(day_vars, 1)
    constraints.append(pulp.lpSum(single_vars.values()) <= max_people_per_day)
    constraints.append(pulp.lpSum(single_vars.values()) >= min_people_per_day)
    
# Some days people cannot come on some days
for empl in employees_ids:
    employee_vars = get_employee_vars_all(x, empl)
    single_vars = get_length_vars(employee_vars, 1)
    for var, avail in zip(single_vars, availabilities[empl]):
        if avail == 0:
            constraints.append(single_vars[var] == 0)
            
# Add target minimum presence constraints
for empl in employees_ids:
    constraints.append(y[str(empl)] <= 0)
    employee_vars = get_employee_vars_all(x, empl)
    single_vars = get_length_vars(employee_vars, 1)
    constraints.append(y[str(empl)] <= - (target_presence - pulp.lpSum(single_vars)))
            
# Linearization of product constraints
for name, value in linearize_vars.items():
    single_vars = [x[y] for y in name.split('__')]
    for single in single_vars:
        constraints.append(value <= single)
    constraints.append(pulp.lpSum(single_vars) <= value + len(single_vars) - 1)

#### Solve

In [104]:
model = pulp.LpProblem("office-returns", pulp.LpMaximize)
model += objective
for const in constraints:
    model += const

In [105]:
%%time
model.solve()
print(pulp.LpStatus[model.status])

Optimal
CPU times: user 48.9 ms, sys: 3.48 ms, total: 52.4 ms
Wall time: 3.38 s


#### Results

In [106]:
single_vars = get_length_vars(x, 1)
res = collections.defaultdict(list)
for key, value in single_vars.items():
    empl = key.split('_')[0]
    day = key.split('_')[1]
    res[empl].append(value.varValue)
    # print(f"Employee number {empl} on day {day}: {value.varValue}")

df = pd.DataFrame(res).T

In [107]:
df.index = employees_names
df.columns = date_range.strftime('%b %d')
display(df.astype(int).style.background_gradient(cmap="Blues"))
df.loc['col_total']= df.sum(axis=0)
df.loc[:,'row_total'] = df.sum(axis=1)
display(df)

,Oct 18,Oct 19,Oct 20,Oct 21,Oct 22,Oct 25,Oct 26,Oct 27,Oct 28,Oct 29,Nov 01,Nov 02,Nov 03,Nov 04,Nov 05,Nov 08,Nov 09,Nov 10,Nov 11,Nov 12
BISTECCA,1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0
BURRO,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
FILETTO,1,1,0,1,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0
FORMAGGIO,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,1,1,1,1
GELATO,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0
INSALATA,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0
LIMONE,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1
ORATA,0,1,1,1,0,0,0,1,1,0,0,1,1,1,0,0,0,1,1,0
SALMONE,0,0,1,1,1,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0
STRUTTO,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1


,Oct 18,Oct 19,Oct 20,Oct 21,Oct 22,Oct 25,Oct 26,Oct 27,Oct 28,Oct 29,Nov 01,Nov 02,Nov 03,Nov 04,Nov 05,Nov 08,Nov 09,Nov 10,Nov 11,Nov 12,row_total
BISTECCA,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.0
BURRO,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0
FILETTO,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
FORMAGGIO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,9.0
GELATO,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,13.0
INSALATA,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,13.0
LIMONE,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,8.0
ORATA,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,10.0
SALMONE,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
STRUTTO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,9.0


In [108]:
available_color = PatternFill(start_color=available, end_color=available, fill_type="solid")
unavailable_color = PatternFill(start_color=unavailable, end_color=unavailable, fill_type="solid")

wb = Workbook()
dest_filename = 'scheduling.xlsx'
ws = wb.active
ws.title = "scheduling"

col_range = len(res[list(res.keys())[0]])
row_range = len(res)

# Write first row
dt_row = list(date_range.date)
dt_row.insert(0, '')
ws.append(dt_row)

for row in range(1, row_range):
    cell = ws.cell(row + 1, 1)
    cell.value = employees_names[row-1]
    for col in range(1, col_range + 1):
        cell = ws.cell(row + 1, col + 1)
        if res[str(row)][col-1] == 0:
            cell.fill = unavailable_color
        else:
            cell.fill = available_color
wb.save(filename = dest_filename)

#### Improvements

* Inserting preference days